In [2]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import statsmodels.api as sm
import geopandas as gpd
import os
import dask
import dask.dataframe as dd
import itertools
from itertools import chain
from math import sqrt, floor, ceil, isnan
import multiprocess
import multiprocessing
import importlib
from importlib import reload
from collections import Counter
from fuzzywuzzy import process, fuzz
import time
import warnings
import datetime
from datetime import datetime
from datetime import date
warnings.filterwarnings("error")

pd.options.display.max_columns = 500
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = 400

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
us_state_to_abbrev = pd.DataFrame.from_dict(us_state_to_abbrev,orient='index').reset_index()
us_state_to_abbrev.columns = ['StateFull','State']

# Obtain mapping of FIPS state code to state name
GOVS_code = pd.read_excel('../RawData/GovFinSurvey/_IndFin_1967-2012/GOVS_to_FIPS_Codes_State_&_County_2012.xls',
    sheet_name='County Codes')
GOVS_code = GOVS_code[GOVS_code['Unnamed: 2']!='code']
GOVS_code = GOVS_code[['Unnamed: 5','Unnamed: 11']]
GOVS_code = GOVS_code[~pd.isnull(GOVS_code['Unnamed: 5'])]
GOVS_code = GOVS_code[~pd.isnull(GOVS_code['Unnamed: 11'])]
GOVS_code = GOVS_code.rename(columns={
    'Unnamed: 5':'FIPS State Code',
    'Unnamed: 11':'StateFull'})
GOVS_code['FIPS State Code'] = GOVS_code['FIPS State Code'].astype(int)
StateFIPSCode = GOVS_code[['FIPS State Code','StateFull']].drop_duplicates()
StateFIPSCode['FIPS State Code'] = StateFIPSCode['FIPS State Code'].astype(int)


# 1. Demographics

Notes:
- For population, use Census Bureau data for 2021-2022, but NIH data piror to that, as some files are missing from Census Bureau data.

## 1.1. Population

### 1.1.1 CSA level

In [20]:
# Data for 2021-2022
CSA_POP_2022 = pd.read_csv("../RawData/MSA/POP/csa-est2022.csv",encoding = "ISO-8859-1")
CSA_POP_2022 = CSA_POP_2022[CSA_POP_2022['LSAD']=='Combined Statistical Area']
CSA_POP_2022 = CSA_POP_2022[['CSA','POPESTIMATE2021','POPESTIMATE2022']].reset_index(drop=True)
CSA_POP_2022 = CSA_POP_2022.rename(columns={'CSA':'CSA Code','POPESTIMATE2021':'pop_2021','POPESTIMATE2022':'pop_2022'})

# Data prior to that
us1969_2020 = pd.read_csv("../RawData/MSA/POP/us.1969_2020.19ages.adjusted.txt",header=None)

us1969_2020['year'] = us1969_2020[0].str.slice(0,4)
us1969_2020['state'] = us1969_2020[0].str.slice(4,6)
us1969_2020['state_FIPS'] = us1969_2020[0].str.slice(6,8)
us1969_2020['county_FIPS'] = us1969_2020[0].str.slice(8,11)
us1969_2020['registry'] = us1969_2020[0].str.slice(11,13)
us1969_2020['race'] = us1969_2020[0].str.slice(13,14)
us1969_2020['origin'] = us1969_2020[0].str.slice(14,15)
us1969_2020['sex'] = us1969_2020[0].str.slice(15,16)
us1969_2020['age'] = us1969_2020[0].str.slice(16,18)
us1969_2020['pop'] = us1969_2020[0].str.slice(18,26).astype(int)
us1969_2020 = us1969_2020.drop(columns=[0])

us1969_2020['med_age_of_group'] = None
us1969_2020.loc[us1969_2020['age']=='00','med_age_of_group'] = 0
us1969_2020.loc[us1969_2020['age']=='01','med_age_of_group'] = 2.5
us1969_2020.loc[us1969_2020['age']=='02','med_age_of_group'] = 7.5
us1969_2020.loc[us1969_2020['age']=='03','med_age_of_group'] = 12.5
us1969_2020.loc[us1969_2020['age']=='04','med_age_of_group'] = 17.5
us1969_2020.loc[us1969_2020['age']=='05','med_age_of_group'] = 22.5
us1969_2020.loc[us1969_2020['age']=='06','med_age_of_group'] = 27.5
us1969_2020.loc[us1969_2020['age']=='07','med_age_of_group'] = 32.5
us1969_2020.loc[us1969_2020['age']=='08','med_age_of_group'] = 37.5
us1969_2020.loc[us1969_2020['age']=='09','med_age_of_group'] = 42.5
us1969_2020.loc[us1969_2020['age']=='10','med_age_of_group'] = 47.5
us1969_2020.loc[us1969_2020['age']=='11','med_age_of_group'] = 52.5
us1969_2020.loc[us1969_2020['age']=='12','med_age_of_group'] = 57.5
us1969_2020.loc[us1969_2020['age']=='13','med_age_of_group'] = 62.5
us1969_2020.loc[us1969_2020['age']=='14','med_age_of_group'] = 67.5
us1969_2020.loc[us1969_2020['age']=='15','med_age_of_group'] = 72.5
us1969_2020.loc[us1969_2020['age']=='16','med_age_of_group'] = 77.5
us1969_2020.loc[us1969_2020['age']=='17','med_age_of_group'] = 82.5
us1969_2020.loc[us1969_2020['age']=='18','med_age_of_group'] = 87.5
us1969_2020['med_age_of_groupXpop'] = us1969_2020['med_age_of_group']*us1969_2020['pop']

us1969_2020['is_minority'] = us1969_2020['race']!='1'
us1969_2020['is_minorityXpop'] = us1969_2020['is_minority']*us1969_2020['pop']

us1969_2020 = us1969_2020.rename(columns={'state_FIPS':'FIPS State Code','county_FIPS':'FIPS County Code'})
us1969_2020 = us1969_2020.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum,'med_age_of_groupXpop':sum,'is_minorityXpop':sum})
us1969_2020 = us1969_2020.reset_index()

us1969_2020['age'] = us1969_2020['med_age_of_groupXpop']/us1969_2020['pop']
us1969_2020['is_minority'] = us1969_2020['is_minorityXpop']/us1969_2020['pop']

CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]
CBSAData['FIPS State Code'] = CBSAData['FIPS State Code'].astype(int).astype(str)
CBSAData['FIPS County Code'] = CBSAData['FIPS County Code'].astype(int).astype(str)
CBSAData.loc[CBSAData['FIPS State Code'].str.len()==1,'FIPS State Code'] = '0'+CBSAData['FIPS State Code'][CBSAData['FIPS State Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==1,'FIPS County Code'] = '00'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==2,'FIPS County Code'] = '0'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==2]

pop_by_CSA = us1969_2020.merge(CBSAData[~pd.isnull(CBSAData['CSA Code'])][['CSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_CSA = pop_by_CSA.groupby(['year','CSA Code']).agg({'pop':sum,'age':'mean','is_minority':'mean'}).reset_index()

# Combine two data sources
pop_by_CSA_2021 = CSA_POP_2022[['CSA Code','pop_2021']].rename(columns={'pop_2021':'pop'})
pop_by_CSA_2021['year'] = 2021
pop_by_CSA_2022 = CSA_POP_2022[['CSA Code','pop_2022']].rename(columns={'pop_2022':'pop'})
pop_by_CSA_2022['year'] = 2022
# Stipulate population of 2023 to be identical as 2022
pop_by_CSA_2023 = pop_by_CSA_2022.copy()
pop_by_CSA_2023['year'] = 2023
pop_by_CSA = pd.concat([pop_by_CSA,pop_by_CSA_2021,pop_by_CSA_2022,pop_by_CSA_2023])

# Growth rate of population
pop_by_CSA['year'] = pop_by_CSA['year'].astype(int)
pop_by_CSA_shiftm1 = pop_by_CSA.copy()
pop_by_CSA_shiftm1['year'] = pop_by_CSA_shiftm1['year']+1
pop_by_CSA_shiftm1 = pop_by_CSA_shiftm1[['year','CSA Code','pop']]
pop_by_CSA_shiftm1 = pop_by_CSA_shiftm1.rename(columns={'pop':'popm1'})
pop_by_CSA = pop_by_CSA.merge(pop_by_CSA_shiftm1,on=['year','CSA Code'],how='outer')
pop_by_CSA['pop_inc_rate'] = pop_by_CSA['pop']/pop_by_CSA['popm1']-1

pop_by_CSA.to_csv("../CleanData/Demographics/0C_CSA_Pop.csv")

### 1.1.2 CBSA level

In [ ]:
##############################
# CBSA-level population size #
##############################

# "CSA_POP_2022" do not have all the CBSAs. Use county level data from "us1969_2020" and stipulate for years after 2020

# Data prior to that
us1969_2020 = pd.read_csv("../RawData/MSA/POP/us.1969_2020.19ages.adjusted.txt",header=None)

us1969_2020['year'] = us1969_2020[0].str.slice(0,4)
us1969_2020['state'] = us1969_2020[0].str.slice(4,6)
us1969_2020['state_FIPS'] = us1969_2020[0].str.slice(6,8)
us1969_2020['county_FIPS'] = us1969_2020[0].str.slice(8,11)
us1969_2020['registry'] = us1969_2020[0].str.slice(11,13)
us1969_2020['race'] = us1969_2020[0].str.slice(13,14)
us1969_2020['origin'] = us1969_2020[0].str.slice(14,15)
us1969_2020['sex'] = us1969_2020[0].str.slice(15,16)
us1969_2020['age'] = us1969_2020[0].str.slice(16,18)
us1969_2020['pop'] = us1969_2020[0].str.slice(18,26).astype(int)
us1969_2020 = us1969_2020.drop(columns=[0])

us1969_2020 = us1969_2020.rename(columns={'state_FIPS':'FIPS State Code','county_FIPS':'FIPS County Code'})
us1969_2020 = us1969_2020.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum})
us1969_2020 = us1969_2020.reset_index()

CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]
CBSAData['FIPS State Code'] = CBSAData['FIPS State Code'].astype(int).astype(str)
CBSAData['FIPS County Code'] = CBSAData['FIPS County Code'].astype(int).astype(str)
CBSAData.loc[CBSAData['FIPS State Code'].str.len()==1,'FIPS State Code'] = '0'+CBSAData['FIPS State Code'][CBSAData['FIPS State Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==1,'FIPS County Code'] = '00'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==2,'FIPS County Code'] = '0'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==2]

pop_by_CBSA = us1969_2020.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_CBSA = pop_by_CBSA.groupby(['year','CBSA Code']).agg({'pop':sum}).reset_index()

pop_by_CBSA['year'] = pop_by_CBSA['year'].astype(int)

# Combine two data sources
pop_by_CBSA_2021 = pop_by_CBSA[pop_by_CBSA['year']==2020].copy()
pop_by_CBSA_2021['year'] = 2021
pop_by_CBSA_2022 = pop_by_CBSA[pop_by_CBSA['year']==2020].copy()
pop_by_CBSA_2022['year'] = 2022
pop_by_CBSA_2023 = pop_by_CBSA[pop_by_CBSA['year']==2020].copy()
pop_by_CBSA_2023['year'] = 2023
pop_by_CBSA = pd.concat([pop_by_CBSA,pop_by_CBSA_2021,pop_by_CBSA_2022,pop_by_CBSA_2023])

pop_by_CBSA.to_csv("../CleanData/Demographics/0C_CBSA_Pop.csv")

### 1.1.3 State level

In [8]:
###############################
# State-level population size #
###############################

# "CSA_POP_2022" do not have all the CBSAs. Use county level data from "us1969_2020" and stipulate for years after 2020

# Data prior to that
us1969_2020 = pd.read_csv("../RawData/MSA/POP/us.1969_2020.19ages.adjusted.txt",header=None)

us1969_2020['year'] = us1969_2020[0].str.slice(0,4)
us1969_2020['state'] = us1969_2020[0].str.slice(4,6)
us1969_2020['state_FIPS'] = us1969_2020[0].str.slice(6,8)
us1969_2020['county_FIPS'] = us1969_2020[0].str.slice(8,11)
us1969_2020['registry'] = us1969_2020[0].str.slice(11,13)
us1969_2020['race'] = us1969_2020[0].str.slice(13,14)
us1969_2020['origin'] = us1969_2020[0].str.slice(14,15)
us1969_2020['sex'] = us1969_2020[0].str.slice(15,16)
us1969_2020['age'] = us1969_2020[0].str.slice(16,18)
us1969_2020['pop'] = us1969_2020[0].str.slice(18,26).astype(int)
us1969_2020 = us1969_2020.drop(columns=[0])

us1969_2020 = us1969_2020.rename(columns={'state_FIPS':'FIPS State Code','county_FIPS':'FIPS County Code'})
us1969_2020 = us1969_2020.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum})
us1969_2020 = us1969_2020.reset_index()

CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]
CBSAData['FIPS State Code'] = CBSAData['FIPS State Code'].astype(int).astype(str)
CBSAData['FIPS County Code'] = CBSAData['FIPS County Code'].astype(int).astype(str)
CBSAData.loc[CBSAData['FIPS State Code'].str.len()==1,'FIPS State Code'] = '0'+CBSAData['FIPS State Code'][CBSAData['FIPS State Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==1,'FIPS County Code'] = '00'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==2,'FIPS County Code'] = '0'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==2]

pop_by_State = us1969_2020.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_State = pop_by_State.groupby(['year','FIPS State Code']).agg({'pop':sum}).reset_index()

pop_by_State['year'] = pop_by_State['year'].astype(int)

# Combine two data sources
pop_by_State_2021 = pop_by_State[pop_by_State['year']==2020].copy()
pop_by_State_2021['year'] = 2021
pop_by_State_2022 = pop_by_State[pop_by_State['year']==2020].copy()
pop_by_State_2022['year'] = 2022
pop_by_State_2023 = pop_by_State[pop_by_State['year']==2020].copy()
pop_by_State_2023['year'] = 2023
pop_by_State = pd.concat([pop_by_State,pop_by_State_2021,pop_by_State_2022,pop_by_State_2023])

# Format state name to a "State" variable of two chars
pop_by_State['FIPS State Code'] = pop_by_State['FIPS State Code'].astype(int)
pop_by_State = pop_by_State.merge(StateFIPSCode,on=['FIPS State Code'])
pop_by_State = pop_by_State.merge(us_state_to_abbrev,on=['StateFull'])

pop_by_State.to_csv("../CleanData/Demographics/0C_State_Pop.csv")

## 1.2 Income

For income, use data from BEA.

In [11]:
CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]
CBSAData['FIPS State Code'] = CBSAData['FIPS State Code'].astype(int).astype(str)
CBSAData['FIPS County Code'] = CBSAData['FIPS County Code'].astype(int).astype(str)
CBSAData.loc[CBSAData['FIPS State Code'].str.len()==1,'FIPS State Code'] = '0'+CBSAData['FIPS State Code'][CBSAData['FIPS State Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==1,'FIPS County Code'] = '00'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==2,'FIPS County Code'] = '0'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==2]


### 1.2.1 CSA level

In [19]:
####################
# CSA-level income #
####################

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
inc_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Per capita personal income (dollars) 2/'].copy()
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace(' ','')
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace('"','')
inc_by_county['FIPS State Code'] = inc_by_county['GeoFIPS'].str[:2]
inc_by_county['FIPS County Code'] = inc_by_county['GeoFIPS'].str[2:5]

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
pop_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Population (persons) 1/'].copy()
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace(' ','')
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace('"','')
pop_by_county['FIPS State Code'] = pop_by_county['GeoFIPS'].str[:2]
pop_by_county['FIPS County Code'] = pop_by_county['GeoFIPS'].str[2:5]


inc_by_county = inc_by_county.merge(CBSAData[~pd.isnull(CBSAData['CSA Code'])][['CSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_county = pop_by_county.merge(CBSAData[~pd.isnull(CBSAData['CSA Code'])][['CSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])

inc_by_CSA = pd.DataFrame()

for year in range(1969,2022):
    
    inc_by_county_oneyear = inc_by_county[[str(year),'CSA Code','FIPS State Code','FIPS County Code']]
    inc_by_county_oneyear = inc_by_county_oneyear.rename(columns={str(year):'inc'})
    inc_by_county_oneyear = inc_by_county_oneyear[inc_by_county_oneyear['inc']!='(NA)']
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['inc'].astype(float)

    pop_by_county_oneyear = pop_by_county[[str(year),'CSA Code','FIPS State Code','FIPS County Code']]
    pop_by_county_oneyear = pop_by_county_oneyear.rename(columns={str(year):'pop'})
    pop_by_county_oneyear = pop_by_county_oneyear[pop_by_county_oneyear['pop']!='(NA)']
    pop_by_county_oneyear['pop'] = pop_by_county_oneyear['pop'].astype(float)
    
    inc_by_county_oneyear = inc_by_county_oneyear.merge(pop_by_county_oneyear,on=['FIPS State Code','FIPS County Code','CSA Code'])

    # Calculate weighted average income
    inc_by_county_oneyear['incXpop'] = inc_by_county_oneyear['inc']*inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear.groupby(['CSA Code']).agg({'pop':sum,'incXpop':sum})
    inc_by_county_oneyear = inc_by_county_oneyear.reset_index()
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['incXpop']/inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear[['inc','CSA Code']]
    inc_by_county_oneyear['year'] = year

    inc_by_CSA = pd.concat([inc_by_CSA,inc_by_county_oneyear])

# Supplement with 2022 and 2023, for which I assume income to be the same as 2021
inc_by_county_oneyear['year'] = 2022
inc_by_CSA = pd.concat([inc_by_CSA,inc_by_county_oneyear])
inc_by_county_oneyear['year'] = 2023
inc_by_CSA = pd.concat([inc_by_CSA,inc_by_county_oneyear])

# Growth rate of income
inc_by_CSA['year'] = inc_by_CSA['year'].astype(int)
inc_by_CSA_shiftm1 = inc_by_CSA.copy()
inc_by_CSA_shiftm1['year'] = inc_by_CSA_shiftm1['year']+1
inc_by_CSA_shiftm1 = inc_by_CSA_shiftm1[['year','CSA Code','inc']]
inc_by_CSA_shiftm1 = inc_by_CSA_shiftm1.rename(columns={'inc':'incm1'})
inc_by_CSA = inc_by_CSA.merge(inc_by_CSA_shiftm1,on=['year','CSA Code'],how='outer')
inc_by_CSA['inc_inc_rate'] = inc_by_CSA['inc']/inc_by_CSA['incm1']-1

inc_by_CSA.to_csv("../CleanData/Demographics/0C_CSA_Inc.csv")

### 1.2.2 County level

In [ ]:
#######################
# County-level income #
#######################

inc_by_county_long = pd.DataFrame()
for year in range(1969,2022):
    
    inc_by_county_oneyear = inc_by_county[[str(year),'CSA Code','FIPS State Code','FIPS County Code']]
    inc_by_county_oneyear = inc_by_county_oneyear.rename(columns={str(year):'inc'})
    inc_by_county_oneyear = inc_by_county_oneyear[inc_by_county_oneyear['inc']!='(NA)']
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['inc'].astype(float)
    inc_by_county_oneyear['year'] = year

    inc_by_county_long = pd.concat([inc_by_county_long,inc_by_county_oneyear])

inc_by_county_long.to_csv("../CleanData/Demographics/0C_County_Inc.csv")

### 1.2.3 CBSA level

In [ ]:
#####################
# CBSA-level income #
#####################

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
inc_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Per capita personal income (dollars) 2/'].copy()
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace(' ','')
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace('"','')
inc_by_county['FIPS State Code'] = inc_by_county['GeoFIPS'].str[:2]
inc_by_county['FIPS County Code'] = inc_by_county['GeoFIPS'].str[2:5]

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
pop_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Population (persons) 1/'].copy()
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace(' ','')
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace('"','')
pop_by_county['FIPS State Code'] = pop_by_county['GeoFIPS'].str[:2]
pop_by_county['FIPS County Code'] = pop_by_county['GeoFIPS'].str[2:5]


inc_by_county = inc_by_county.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_county = pop_by_county.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])

inc_by_CBSA = pd.DataFrame()

for year in range(1969,2022):
    
    inc_by_county_oneyear = inc_by_county[[str(year),'CBSA Code','FIPS State Code','FIPS County Code']]
    inc_by_county_oneyear = inc_by_county_oneyear.rename(columns={str(year):'inc'})
    inc_by_county_oneyear = inc_by_county_oneyear[inc_by_county_oneyear['inc']!='(NA)']
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['inc'].astype(float)

    pop_by_county_oneyear = pop_by_county[[str(year),'CBSA Code','FIPS State Code','FIPS County Code']]
    pop_by_county_oneyear = pop_by_county_oneyear.rename(columns={str(year):'pop'})
    pop_by_county_oneyear = pop_by_county_oneyear[pop_by_county_oneyear['pop']!='(NA)']
    pop_by_county_oneyear['pop'] = pop_by_county_oneyear['pop'].astype(float)
    
    inc_by_county_oneyear = inc_by_county_oneyear.merge(pop_by_county_oneyear,on=['FIPS State Code','FIPS County Code','CBSA Code'])

    # Calculate weighted average income
    inc_by_county_oneyear['incXpop'] = inc_by_county_oneyear['inc']*inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear.groupby(['CBSA Code']).agg({'pop':sum,'incXpop':sum})
    inc_by_county_oneyear = inc_by_county_oneyear.reset_index()
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['incXpop']/inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear[['inc','CBSA Code']]
    inc_by_county_oneyear['year'] = year

    inc_by_CBSA = pd.concat([inc_by_CBSA,inc_by_county_oneyear])

# Supplement with 2022 and 2023, for which I assume income to be the same as 2021
inc_by_county_oneyear['year'] = 2022
inc_by_CBSA = pd.concat([inc_by_CBSA,inc_by_county_oneyear])
inc_by_county_oneyear['year'] = 2023
inc_by_CBSA = pd.concat([inc_by_CBSA,inc_by_county_oneyear])

inc_by_CBSA.to_csv("../CleanData/Demographics/0C_CBSA_Inc.csv")

### 1.2.4 State level

In [15]:
######################
# State-level income #
######################

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
inc_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Per capita personal income (dollars) 2/'].copy()
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace(' ','')
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace('"','')
inc_by_county['FIPS State Code'] = inc_by_county['GeoFIPS'].str[:2]
inc_by_county['FIPS County Code'] = inc_by_county['GeoFIPS'].str[2:5]

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
pop_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Population (persons) 1/'].copy()
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace(' ','')
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace('"','')
pop_by_county['FIPS State Code'] = pop_by_county['GeoFIPS'].str[:2]
pop_by_county['FIPS County Code'] = pop_by_county['GeoFIPS'].str[2:5]


inc_by_county = inc_by_county.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_county = pop_by_county.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])

inc_by_State = pd.DataFrame()

for year in range(1969,2022):
    
    inc_by_county_oneyear = inc_by_county[[str(year),'CBSA Code','FIPS State Code','FIPS County Code']]
    inc_by_county_oneyear = inc_by_county_oneyear.rename(columns={str(year):'inc'})
    inc_by_county_oneyear = inc_by_county_oneyear[inc_by_county_oneyear['inc']!='(NA)']
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['inc'].astype(float)

    pop_by_county_oneyear = pop_by_county[[str(year),'CBSA Code','FIPS State Code','FIPS County Code']]
    pop_by_county_oneyear = pop_by_county_oneyear.rename(columns={str(year):'pop'})
    pop_by_county_oneyear = pop_by_county_oneyear[pop_by_county_oneyear['pop']!='(NA)']
    pop_by_county_oneyear['pop'] = pop_by_county_oneyear['pop'].astype(float)
    
    inc_by_county_oneyear = inc_by_county_oneyear.merge(pop_by_county_oneyear,on=['FIPS State Code','FIPS County Code','CBSA Code'])

    # Calculate weighted average income
    inc_by_county_oneyear['incXpop'] = inc_by_county_oneyear['inc']*inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear.groupby(['FIPS State Code']).agg({'pop':sum,'incXpop':sum})
    inc_by_county_oneyear = inc_by_county_oneyear.reset_index()
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['incXpop']/inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear[['inc','FIPS State Code']]
    inc_by_county_oneyear['year'] = year

    inc_by_State = pd.concat([inc_by_State,inc_by_county_oneyear])

# Supplement with 2022 and 2023, for which I assume income to be the same as 2021
inc_by_county_oneyear['year'] = 2022
inc_by_State = pd.concat([inc_by_State,inc_by_county_oneyear])
inc_by_county_oneyear['year'] = 2023
inc_by_State = pd.concat([inc_by_State,inc_by_county_oneyear])


# Format state name to a "State" variable of two chars
inc_by_State['FIPS State Code'] = inc_by_State['FIPS State Code'].astype(int)
inc_by_State = inc_by_State.merge(StateFIPSCode,on=['FIPS State Code'])
inc_by_State = inc_by_State.merge(us_state_to_abbrev,on=['StateFull'])

inc_by_State.to_csv("../CleanData/Demographics/0C_State_Inc.csv")

## 1.3 County-level composite data

Notes:
- Include population,income,and Black ratio.

In [ ]:
# Data prior to that
us1969_2020 = pd.read_csv("../RawData/MSA/POP/us.1969_2020.19ages.adjusted.txt",header=None)

us1969_2020['year'] = us1969_2020[0].str.slice(0,4)
us1969_2020['state'] = us1969_2020[0].str.slice(4,6)
us1969_2020['state_FIPS'] = us1969_2020[0].str.slice(6,8)
us1969_2020['county_FIPS'] = us1969_2020[0].str.slice(8,11)
us1969_2020['registry'] = us1969_2020[0].str.slice(11,13)
us1969_2020['race'] = us1969_2020[0].str.slice(13,14)
us1969_2020['origin'] = us1969_2020[0].str.slice(14,15)
us1969_2020['sex'] = us1969_2020[0].str.slice(15,16)
us1969_2020['age'] = us1969_2020[0].str.slice(16,18)
us1969_2020['pop'] = us1969_2020[0].str.slice(18,26).astype(int)
us1969_2020 = us1969_2020.drop(columns=[0])

us1969_2020 = us1969_2020.rename(columns={'state_FIPS':'FIPS State Code','county_FIPS':'FIPS County Code'})

# Calculate black ratio
black_pop = us1969_2020[us1969_2020['race']=='2'][['year','FIPS State Code','FIPS County Code','pop']]
black_pop = black_pop.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum})
black_pop = black_pop.rename(columns={'pop':'black_pop'})

county_pop = us1969_2020.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum})
county_pop = county_pop.reset_index()

black_pop = black_pop.merge(county_pop,on=['FIPS State Code','FIPS County Code','year'])
black_pop['black_ratio'] = black_pop['black_pop']/black_pop['pop']

# Income by county, including those not in CBSA
CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
inc_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Per capita personal income (dollars) 2/'].copy()
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace(' ','')
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace('"','')
inc_by_county['FIPS State Code'] = inc_by_county['GeoFIPS'].str[:2]
inc_by_county['FIPS County Code'] = inc_by_county['GeoFIPS'].str[2:5]

inc = pd.DataFrame()
for year in range(1969,2021):
    inc_oneyear = inc_by_county[['FIPS State Code','FIPS County Code',str(year)]].copy()
    inc_oneyear = inc_oneyear.rename(columns={str(year):'inc'})
    inc_oneyear['year'] = year
    inc = pd.concat([inc,inc_oneyear])

# Note that CBSA should not be used below: It is not a complete list of all counties in the US

# Complete list of counties, including those not part of CSA 
all_counties = pd.read_csv("../RawData/MSA/fips-by-state.csv",sep=',',encoding="ISO-8859-1",low_memory=False)
all_counties = all_counties.rename(columns={'name':'County','state':'State'})
all_counties['County'] = all_counties['County'].str.upper()
all_counties['County'] = all_counties['County'].str.replace(' COUNTY','')
all_counties['County'] = all_counties['County'].str.replace(' AND ',' & ')
all_counties['County'] = all_counties['County'].str.replace('.','',regex=False)

all_counties['fips'] = all_counties['fips'].astype(str)
all_counties.loc[all_counties['fips'].str.len()==4,'fips'] = '0'+all_counties['fips'][all_counties['fips'].str.len()==4]
all_counties['FIPS State Code'] = all_counties['fips'].str.slice(0,2)
all_counties['FIPS County Code'] = all_counties['fips'].str.slice(2,5)

all_counties_Exploded = pd.DataFrame()
for year in range(1969,2021):
    all_counties['year'] = year
    all_counties_Exploded = pd.concat([all_counties_Exploded,all_counties])
all_counties_Exploded['year'] = all_counties_Exploded['year'].astype(str)

# Merge with county data
black_pop = all_counties_Exploded.merge(black_pop,on=['FIPS State Code','FIPS County Code','year'])
black_pop['year'] = black_pop['year'].astype(int)
black_pop = black_pop.merge(inc,on=['FIPS State Code','FIPS County Code','year'])
black_pop.loc[black_pop['inc']=='(NA)','inc'] = None
black_pop['inc'] = black_pop['inc'].astype(float)

# Export data
black_pop['year'] = black_pop['year'].astype(int)
black_pop_2021 = black_pop[black_pop['year']==2020].copy()
black_pop_2021['year'] = 2021
black_pop_2022 = black_pop[black_pop['year']==2020].copy()
black_pop_2022['year'] = 2022
black_pop_2023 = black_pop[black_pop['year']==2020].copy()
black_pop_2023['year'] = 2023
black_pop = pd.concat([black_pop,black_pop_2021,black_pop_2022,black_pop_2023])
black_pop.to_csv("../CleanData/Demographics/0C_County_Composite.csv")

# 2. Geographics

Notes:
- Get a list of MSAs that has overlap in terms of being in the same state. Use these to rule out unreasonable control groups that might also get affected by the treatment. Note that just by matching using the population and average income will result in some treated-control pairs that are very geographically approximate.

In [ ]:
# "CSA" is for metropolitan and "CBSAData" includes also those micropolitan
CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]

# Add state abbreviations
%run -i SCRIPT_us_states.py
us_state_to_abbrev = pd.DataFrame.from_dict(us_state_to_abbrev,orient='index').reset_index()
us_state_to_abbrev.columns = ['State Name','State']
CBSAData = CBSAData.rename(columns={'County/County Equivalent':'County'})
CBSAData = CBSAData.merge(us_state_to_abbrev,on='State Name',how='outer',indicator=True)
CBSAData = CBSAData[CBSAData['_merge']=='both'].drop(columns=['_merge'])
# Merge is perfect
CBSAData['County'] = CBSAData['County'].str.upper().str.replace(' COUNTY','')


#-----------#
# CSA pairs #
#-----------#

Same_State_CSA_pairs = []
CSAs = list(CBSAData['CSA Code'].unique())
CSAs = [item for item in CSAs if item!=None and str(item)!='nan']
for CSA in CSAs:
    # A list of CSAs in the same state
    States = list(CBSAData[CBSAData['CSA Code']==CSA]['FIPS State Code'].unique())
    for State in States:
        CSAs_same_state = list(CBSAData[CBSAData['FIPS State Code']==State]['CSA Code'].unique())
        CSAs_same_state = [item for item in CSAs_same_state if item!=None and str(item)!='nan' and item!=CSA]
        for item in CSAs_same_state:
            Same_State_CSA_pairs = Same_State_CSA_pairs+[{'CSA_1':CSA,'CSA_2':item}]
            Same_State_CSA_pairs = Same_State_CSA_pairs+[{'CSA_1':item,'CSA_2':CSA}]

# Pairs of CSAs in the same state
Title_Code = CBSAData[['CSA Code','CSA Title']].drop_duplicates()
Title_Code = Title_Code[~pd.isnull(Title_Code['CSA Code'])]
Same_State_CSA_pairs = pd.DataFrame(Same_State_CSA_pairs)
Same_State_CSA_pairs = Same_State_CSA_pairs.merge(Title_Code.rename(columns={'CSA Code':'CSA_1','CSA Title':'Title_1'}),on='CSA_1')
Same_State_CSA_pairs = Same_State_CSA_pairs.merge(Title_Code.rename(columns={'CSA Code':'CSA_2','CSA Title':'Title_2'}),on='CSA_2')

Same_State_CSA_pairs.to_csv("../CleanData/Demographics/0C_Same_State_CSA_pairs.csv")

#------------#
# CBSA pairs #
#------------#

Same_State_CBSA_pairs = []
CBSAs = list(CBSAData['CBSA Code'].unique())
CBSAs = [item for item in CBSAs if item!=None and str(item)!='nan']
for CBSA in CBSAs:
    # A list of CBSAs in the same state
    States = list(CBSAData[CBSAData['CBSA Code']==CBSA]['FIPS State Code'].unique())
    for State in States:
        CBSAs_same_state = list(CBSAData[CBSAData['FIPS State Code']==State]['CBSA Code'].unique())
        CBSAs_same_state = [item for item in CBSAs_same_state if item!=None and str(item)!='nan' and item!=CBSA]
        for item in CBSAs_same_state:
            Same_State_CBSA_pairs = Same_State_CBSA_pairs+[{'CBSA_1':CBSA,'CBSA_2':item}]
            Same_State_CBSA_pairs = Same_State_CBSA_pairs+[{'CBSA_1':item,'CBSA_2':CBSA}]

# Pairs of CBSAs in the same state
Title_Code = CBSAData[['CBSA Code','CBSA Title']].drop_duplicates()
Title_Code = Title_Code[~pd.isnull(Title_Code['CBSA Code'])]
Same_State_CBSA_pairs = pd.DataFrame(Same_State_CBSA_pairs)
Same_State_CBSA_pairs = Same_State_CBSA_pairs.merge(Title_Code.rename(columns={'CBSA Code':'CBSA_1','CBSA Title':'Title_1'}),on='CBSA_1')
Same_State_CBSA_pairs = Same_State_CBSA_pairs.merge(Title_Code.rename(columns={'CBSA Code':'CBSA_2','CBSA Title':'Title_2'}),on='CBSA_2')

Same_State_CBSA_pairs.to_csv("../CleanData/Demographics/0C_Same_State_CBSA_pairs.csv")


## 2.1 Figure out CSA adjacency, which will be used for Sunderam and Scharfstein test

In [62]:
# "CSA" is for metropolitan and "CBSAData" includes also those micropolitan
CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]

CBSAData['StateFull'] = CBSAData['State Name']
CBSAData = CBSAData.merge(us_state_to_abbrev,on=['StateFull'])

CBSAData = CBSAData[['CSA Code','County/County Equivalent','State']]

In [63]:
# Merge in CSA info into county adjancency files
county_adjacency2024 = pd.read_csv("../RawData/MSA/county_adjacency2024.txt",delimiter='|')

county_adjacency2024[['CountyA','StateA']] = county_adjacency2024['County Name'].str.split(',',expand=True)
county_adjacency2024[['CountyB','StateB']] = county_adjacency2024['Neighbor Name'].str.split(',',expand=True)

county_adjacency2024 = county_adjacency2024[['CountyA','StateA','CountyB','StateB']]

county_adjacency2024['StateA'] = county_adjacency2024['StateA'].str.strip()
county_adjacency2024['StateB'] = county_adjacency2024['StateB'].str.strip()

CBSAData_temp = CBSAData.rename(columns={'County/County Equivalent':'CountyA','State':'StateA'})
county_adjacency2024 = county_adjacency2024.merge(CBSAData_temp,on=['CountyA','StateA'])
county_adjacency2024 = county_adjacency2024.rename(columns={'CSA Code':'CSA Code A'})

CBSAData_temp = CBSAData.rename(columns={'County/County Equivalent':'CountyB','State':'StateB'})
county_adjacency2024 = county_adjacency2024.merge(CBSAData_temp,on=['CountyB','StateB'])
county_adjacency2024 = county_adjacency2024.rename(columns={'CSA Code':'CSA Code B'})

# Export a list of neighboring CSAs
county_adjacency2024 = county_adjacency2024[['CSA Code A','CSA Code B']].drop_duplicates()

county_adjacency2024 = county_adjacency2024[county_adjacency2024['CSA Code A']!=county_adjacency2024['CSA Code B']]
county_adjacency2024 = county_adjacency2024[~pd.isnull(county_adjacency2024['CSA Code A'])]
county_adjacency2024 = county_adjacency2024[~pd.isnull(county_adjacency2024['CSA Code B'])]

county_adjacency2024.to_csv('../CleanData/Demographics/CSA_Neighboring.csv')